In [2]:
import pandas as pd
from random import random, randint

In [3]:
df = pd.read_csv('South_Sudan_States.csv')

In [9]:
count_counties = len(df.County.unique())
count_states = len(df.State.unique())
print('There are {0} total states and {1} counties in South Sudan'.format(count_states, count_counties))

There are 29 total states and 211 counties in South Sudan


In [10]:
counties = {}
for county in df.County.unique():
    counties[county] = {'rainfall': random(), 'population': random(), 'infrastructure': random()}

In [186]:
def gen_year(state, county, year):
    obj = {}
    obj['State'] = state
    obj['County'] = county
    obj['Year'] = year
    
    county_obj = counties[county]
    rain_adj = county_obj['rainfall']
    pop_adj = county_obj['population']
    inf_adj = county_obj['infrastructure']
    
    obj['Rainfall'] = round(rain_adj*70 + random()*10,2)
    obj['Population'] = int(round(pop_adj*60000 + random()*5000,0))
    obj['Infrastructure'] = int(round(inf_adj*500 + random()*100,0))
    
    return obj

In [187]:
years = range(2000, 2019)

generated = []

for kk, vv in df.iterrows():
    for i in years:
        obj = gen_year(vv.State, vv.County, i)
        generated.append(obj)

In [188]:
output = pd.DataFrame.from_records(generated)
output = output[['Year','State','County','Population','Rainfall','Infrastructure']]

In [189]:
output.to_csv('../Class Files/Scenario 4/South_Sudan_Regional_Report.csv', index=False)

# Checking

In [191]:
rain = pd.DataFrame(output.groupby('County').Rainfall.mean())
pop = pd.DataFrame(output.groupby('County').Population.mean())
pop = pop.reset_index()
pop = pop.set_index('County')

In [192]:
rain = rain.reset_index().sort_values('Rainfall',ascending=False)
comb = rain.join(pop, on='County')

In [193]:
rain.describe()

,Rainfall
count,211.000000
mean,39.657294
std,20.726121
min,4.110526
25%,20.632895
50%,39.277895
75%,56.819211
max,75.011053


In [194]:
comb[(comb['Rainfall'] > 70) & (comb['Population'] > 48556)]

,County,Rainfall,Population
60,Geria,74.308421,49468.157895
156,Panyok,74.118947,53417.842105
56,Duony,73.387895,62265.578947
192,Waat,73.146316,56979.157895
89,Korok East,73.094737,49745.789474
84,Kodok,72.533684,51814.368421
17,Anyar Nguan,72.401053,51728.210526
20,Apuk South,70.311053,54870.842105


In [195]:
comb[(comb['Rainfall'] > 70)]

,County,Rainfall,Population
125,Manyang-Ngok,75.011053,10544.157895
60,Geria,74.308421,49468.157895
156,Panyok,74.118947,53417.842105
99,Lakama'di County,73.642632,5619.368421
56,Duony,73.387895,62265.578947
192,Waat,73.146316,56979.157895
89,Korok East,73.094737,49745.789474
97,Lafon,73.015263,18752.842105
197,Witto County,72.732632,22398.000000
84,Kodok,72.533684,51814.368421
